In [7]:
# Install the required packages
%pip install pandas Office365-REST-Python-Client
%pip install pyyaml

Note: you may need to restart the kernel to use updated packages.



In [26]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [27]:
# Import the required libraries
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
import io
import pandas as pd

# Import the utils library
import sys
sys.path.append("..")
from utils.filesystem import *


In [36]:
import yaml

config_file = '../config.yaml'

# Load configuration file
with open(config_file, 'r') as config_file:
    config = yaml.safe_load(config_file)

# Set the site URL
site_url = config['site_url']

# Set the username and password
username = config['username']
password = config['password']

# Set the Excel file path
input_file_path = config['file_path']

# Set the Excel local file path
local_path = config['local_path']



In [37]:
# Load Excel data into DataFrame
try:
    # Get list of all files in the input folder
    list_of_files = list_files_in_directory(local_file)

    # Create a dataframe from each Excel file located in list_of_files
    for file in list_of_files:
        # Load all Excel file tabs into a dictionary, where each key is a tab name
        excel_dict = load_excel_to_dict(local_file, file)

        # Remove the extension from the file name
        file_name = os.path.splitext(file)[0]
        # Create a folder name based on the file name and the output folder
        full_filename_path = create_foldername(local_file, file_name)

        # Save each tab of the Excel file as a Parquet file
        save_dict_to_parquet(excel_dict, full_filename_path)

except Exception as e:
    print(e)


No files found in C:/Users/perocha/OneDrive/Documents/Rett/AESR/Tesorero/2023/EXTRACTO BANCO 2024..xlsx
